# Optimization Game with Multiple Facilities and Warehouses

## Overview

In this interactive game, the objective is to guess the optimal location for a new facility based on a set of randomly generated warehouses and their respective demands. The game leverages the Gurobi optimizer to calculate the optimal locations for facilities, minimizing the total Euclidean distance between facilities and the warehouses they serve. The game visually represents warehouses, facilities, and optimal locations with different images to enhance the gaming experience. 

## Key Features

1. **Multiple Facilities & Warehouses**: The game supports a scenario with multiple facilities and warehouses, facilitating a complex and realistic simulation.
2. **Euclidean Distance Optimization**: The script uses the Gurobi optimizer to determine the optimal locations for facilities, minimizing the total Euclidean distance to the warehouses they service.
3. **Visual Insights**: Utilizes images to represent warehouses, facilities, and optimal locations, providing a graphical insight into the optimization problem.
4. **Interactive Gameplay**: Players can interactively guess the optimal location of a new facility. The score is calculated based on the proximity of the guess to the actual optimal location determined by the optimization algorithm.

## Usage

1. **Initializing the Game**: Execute the script to initialize the game. It will generate a random set of warehouse locations and their respective demands, and determine the optimal facility locations based on the current setup.
2. **Playing the Game**: Players can click on the plot to guess the optimal location for a new facility. The score, based on the proximity of the guess to the actual optimal location, will be displayed.
3. **Next Round**: Click on the plot again to initiate a new round with fresh warehouse locations and demands.

## Code Structure

- `game_state`: A dictionary holding the current state of the game, including warehouse locations, demands, and facility details.
- `initialize_game_state()`: A function to set up a new game state with random warehouse locations and demands.
- `find_optimal_facility_locations()`: A function that employs the Gurobi optimizer to find the optimal facility locations based on current warehouse locations and demands.
- `calculate_score(user_x, user_y, optimal_x, optimal_y)`: Computes the score based on the proximity of the user's guess to the optimal location.
- `draw_supply_lines(facility_index)`: A function to draw lines from a facility to the warehouses it supplies, with line darkness indicating the quantity of supply.
- `on_click(event)`: An event handler for click events, which manages the user's guesses and initiates new rounds.

### Customization

- Modify the number of facilities and the range of warehouses and demands in the `game_state` dictionary.
- Adjust the plot window and marker sizes in the `fig, ax = plt.subplots(figsize=(10,10))` line and the marker size parameters in the `ax.plot()` calls.

## Dependencies

- numpy
- gurobipy
- matplotlib

## Execution

To engage in the game, run the script in a Python environment where the required packages are installed. Make guesses by clicking on the plot. The game will automatically progress, providing feedback on your guesses.

## Legend Representation

- **Warehouse**: Represented by the image loaded from 'warehouse.png'. It marks the location of a warehouse on the plot.
- **Facility**: Depicted by the image loaded from 'facility.png'. It indicates the location of a facility on the plot.
- **Optimal Location**: Illustrated by the image loaded from 'optimal1.png'. It signifies the optimal location for a new facility in the game.
- **Supply Line**: The lines connecting facilities and warehouses denote the supply lines. A darker line implies a higher quantity of supply from the facility to the warehouse.


In [1]:
##FUNCTIONING MULTIPLE FACILITIES, MULTIPLE WAREHOUSES, EUCLIDEAN DISTANCE, OPTIMIZATION
import numpy as np
from gurobipy import Model, GRB
%matplotlib tk
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from matplotlib.lines import Line2D


#image loading/logic
def getImage(path, zoom):
    return OffsetImage(plt.imread(path), zoom=zoom)
warehouse_img = getImage('warehouse.png', 0.02)
facility_img = getImage('facility.png', 0.025)
optimal_img = getImage('optimal1.png', 0.03)
optimal_img_small = getImage('optimal1.png', 0.025)



# Variable to keep track of the game state
game_state = {
    "click_count": 0, 
    "number_of_facilities": 3,
    "warehouse_locations": None, 
    "demands": None,
    "facility_locations": None,
    "facility_supply_distribution": None,
    "facility_service_details": None
}

# Function to initialize a new game state
def initialize_game_state():
    warehouse_num = np.random.randint(16, 22)
    game_state["warehouse_locations"] = np.random.rand(warehouse_num, 2) * 100
    game_state["demands"] = np.random.randint(10, 101, warehouse_num)
    game_state["click_count"] = 0
    
    # Clear the existing plot and set limits
    ax.clear()
    ax.set_xlim(-10, 110)
    ax.set_ylim(-10, 110)
    
    ax.set_facecolor('#f0f0f0')
    ax.set_xticks([])
    ax.set_yticks([])
    
    
    
#     cmap = plt.get_cmap("YlOrRd")  # Yellow to Red colormap
#     for i, (x, y) in enumerate(game_state["warehouse_locations"]):
#         demand = game_state["demands"][i]
#         color = cmap((demand - 10) / 90)  # Normalize the demand to range [0, 1]
#         ax.plot(x, y, 's', color=color)  # Plot warehouse with color based on demand

    # Displays the warehouses
    for x, y in game_state["warehouse_locations"]:
        ax.add_artist(AnnotationBbox(warehouse_img, (x, y), frameon=False))

        
    
    # Display demands under the warehouses
    for i, (x, y) in enumerate(game_state["warehouse_locations"]):
        ax.text(x, y - 2, str(game_state["demands"][i]), color='black', fontsize=10, ha='center', va='top')
    
    # Call the new optimization function to find facility locations and supply distribution
    find_optimal_facility_locations()
    
    
    for j, (fx, fy) in enumerate(game_state["facility_locations"][1:]):  # Skip the first facility
        ax.add_artist(AnnotationBbox(facility_img, (fx, fy), frameon=False))
          
    
        # Create a new axes at the top left corner of the window
    legend_ax = fig.add_axes([0.01, 0.8, 0.5, 0.2])
    legend_ax.axis('off')

    # Create legend elements using the imported images
    legend_elements = [
        (warehouse_img, 'Warehouse'),
        (facility_img, 'Facility'),
        (optimal_img_small, 'Optimal Location')
    ]

    # Add the images and labels to the legend in two columns
    legend_ax.add_artist(AnnotationBbox(legend_elements[0][0], (0.05, 0.9), frameon=False))
    legend_ax.text(0.15, 0.9, legend_elements[0][1], verticalalignment='center')

    legend_ax.add_artist(AnnotationBbox(legend_elements[1][0], (0.05, 0.6), frameon=False))
    legend_ax.text(0.15, 0.6, legend_elements[1][1], verticalalignment='center')

    legend_ax.add_artist(AnnotationBbox(legend_elements[2][0], (0.55, 0.9), frameon=False))
    legend_ax.text(0.65, 0.9, legend_elements[2][1], verticalalignment='center')

    # Add a text label to describe the supply line representation
    legend_ax.text(0.55, 0.6, 'Supply Line (Darker = More Supply)', color='black', verticalalignment='center')

        
    
    plt.draw()

# New optimization function: find_optimal_facility_locations (as defined earlier in our conversation)
def find_optimal_facility_locations():
    # Accessing game_state without global keyword
    warehouse_locations = game_state["warehouse_locations"]
    demands = game_state["demands"]
    num_facilities = game_state["number_of_facilities"]
    # Calculate total demand and facility supply capacity
    total_demand = sum(demands)
    facility_supply_capacity = total_demand / num_facilities

    # Create a Gurobi model
    m = Model()

    # Define a grid of potential facility locations
    grid_size = 10
    facility_potential_locations = [(x, y) for x in range(0, 101, grid_size) for y in range(0, 101, grid_size)]


    # Pre-calculate distances from potential facility locations to warehouses
    distances = [[((wx - fx)**2 + (wy - fy)**2)**0.5 for wx, wy in warehouse_locations] for fx, fy in facility_potential_locations]

    # plot a red point at each potential facility location
    for x, y in facility_potential_locations:
        ax.plot(x, y, 'ro', markersize=2)

    # Define Variables
    facility_vars = [m.addVar(vtype=GRB.BINARY, name=f"f_{j}") for j in range(len(facility_potential_locations))]
    supply_vars = [[m.addVar(vtype=GRB.CONTINUOUS, lb=0, name=f"s_{i}_{j}") for j in range(len(facility_potential_locations))] for i in range(len(warehouse_locations))]

    # Define Objective Function
    obj = sum(supply_vars[i][j] * distances[j][i] for i in range(len(warehouse_locations)) for j in range(len(facility_potential_locations)))
    m.setObjective(obj, GRB.MINIMIZE)

    # Define Constraints
    for i in range(len(warehouse_locations)):
        m.addConstr(sum(supply_vars[i][j] for j in range(len(facility_potential_locations))) == demands[i])

    for j in range(len(facility_potential_locations)):
        m.addConstr(sum(supply_vars[i][j] for i in range(len(warehouse_locations))) <= facility_supply_capacity * facility_vars[j])

    # Facility selection constraint: exactly num_facilities should be selected
    m.addConstr(sum(facility_vars) == num_facilities)

    # Optimize Model
    m.optimize()

    # Extract Results and Update game_state
    facility_locations = [(x, y) for j, (x, y) in enumerate(facility_potential_locations) if facility_vars[j].X > 0.5]
    game_state["facility_locations"] = facility_locations

    facility_supply_distribution = [[supply_vars[i][j].X for j in range(len(facility_potential_locations))] for i in range(len(warehouse_locations))]
    
    game_state["facility_supply_distribution"] = facility_supply_distribution
    
    # Update game_state with facility service details
    facility_service_details = []
    for j, (x, y) in enumerate(facility_locations):
        servicing_details = [(i, facility_supply_distribution[i][facility_potential_locations.index((x, y))]) for i in range(len(warehouse_locations)) if facility_supply_distribution[i][facility_potential_locations.index((x, y))] > 0]
        facility_service_details.append(servicing_details)
    game_state["facility_service_details"] = facility_service_details


    # Print the results in the specified format
    for j, (x, y) in enumerate(facility_locations):
        servicing_details = [(i, facility_supply_distribution[i][facility_potential_locations.index((x, y))]) for i in range(len(warehouse_locations)) if facility_supply_distribution[i][facility_potential_locations.index((x, y))] > 0]
        print(f"Facility {j+1} location: ({x}, {y}) servicing: {servicing_details}")

    
    return "success!"

# Function to calculate score
def calculate_score(user_x, user_y, optimal_x, optimal_y):
    distance = np.sqrt((user_x - optimal_x)**2 + (user_y - optimal_y)**2)
    score = min(1, 1 / max(1, distance))
    return score

# Create a plot and connect the event handler
fig, ax = plt.subplots(figsize=(10,10))


def draw_supply_lines(facility_index):
    """Draws lines from a facility to warehouses it supplies.

    Args:
        facility_index (int): Index of the facility in the game_state facility service details list.
    """
    facility_x, facility_y = game_state["facility_locations"][facility_index]
    servicing_details = game_state["facility_service_details"][facility_index]
    
    # Choose a colormap based on whether the facility is new or existing
    if facility_index == 0:
        cmap = plt.get_cmap("binary")  # Cool colors for the new facility
    else:
        cmap = plt.get_cmap("binary")   # Warm colors for existing facilities
    
    max_supply = max(servicing_details, key=lambda x: x[1])[1] if servicing_details else 1

    for warehouse_index, supply_amount in servicing_details:
        warehouse_x, warehouse_y = game_state["warehouse_locations"][warehouse_index]
        color = cmap(supply_amount / max_supply)
        ax.plot([facility_x, warehouse_x], [facility_y, warehouse_y], color=color)


# Event handler for click events
def on_click(event):
    
    game_state["click_count"] += 1
    
    cmap = plt.get_cmap("YlOrRd")  # Yellow to Red colormap
    
    if game_state["click_count"] % 2 == 1:
        user_x, user_y = event.xdata, event.ydata

        # Plot user-selected location
        ax.plot(user_x, user_y, 'go', markersize=15)  # Plot user guess as green circle

        # Find the optimal location for the first facility
        optimal_x, optimal_y = game_state["facility_locations"][0]
        
        # Calculate the score
        score = calculate_score(user_x, user_y, optimal_x, optimal_y)
        
        # Set the score as the title of the plot
        ax.set_title(f"Score: {score:.2f}")

        # Plot optimal location of the first facility and its corresponding paths
        ax.add_artist(AnnotationBbox(optimal_img, (optimal_x, optimal_y), frameon=False))
        
        #         ax.plot(optimal_x, optimal_y, 'bo', markersize=15)  # Plot optimal location as blue circle

        
        draw_supply_lines(0)
        
        for i in range(1, game_state["number_of_facilities"]):
            draw_supply_lines(i)
        
        plt.draw()
    else:
        # Reset the game state and start a new game
        initialize_game_state()

# Initialize the first game state
initialize_game_state()

# Connect the event handler
cid = fig.canvas.mpl_connect('button_press_event', on_click)




Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-24
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 141 rows, 2420 columns and 4840 nonzeros
Model fingerprint: 0xf4b51a4c
Variable types: 2299 continuous, 121 integer (121 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [9e-01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 1e+02]
Presolve time: 0.01s
Presolved: 141 rows, 2420 columns, 4840 nonzeros
Variable types: 2299 continuous, 121 integer (121 binary)
Found heuristic solution: objective 82195.323452
Found heuristic solution: objective 60716.168572

Root relaxation: objective 3.727508e+03, 18 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbe